<a href="https://colab.research.google.com/github/BelloBer/Landslide-Detection/blob/main/Complete3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Complete3

In [7]:
# Improved Landslide Detection Model
# Optimized for better F1 scores, precision, and accuracy

from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Dense, Flatten, Conv2D, MaxPooling2D, Dropout,
                                   BatchNormalization, Input, GlobalAveragePooling2D,
                                   SeparableConv2D, Add, Activation, MultiHeadAttention,
                                   LayerNormalization, Reshape, Concatenate)
from tensorflow.keras import backend as K
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (ModelCheckpoint, EarlyStopping, ReduceLROnPlateau,
                                       TensorBoard, LearningRateScheduler)
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.regularizers import l2
import datetime
import gc
from collections import Counter

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Enable mixed precision for better performance
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

# Define paths
train_data_path = '/content/drive/My Drive/train_data/'
test_data_path = '/content/drive/My Drive/test_data/'
train_csv = "/content/drive/My Drive/train_data/Train.csv"
test_csv = "/content/drive/My Drive/test_data/Test.csv"

# Load and analyze data
train_df = pd.read_csv(train_csv)
print("Dataset Info:")
print(f"Total samples: {len(train_df)}")
print(f"Class distribution:\n{train_df['label'].value_counts()}")
print(f"Class ratio: {train_df['label'].value_counts()[1] / train_df['label'].value_counts()[0]:.3f}")

# Enhanced image loading with better preprocessing
def load_and_preprocess_image(image_id, folder_path, target_size=(256, 256)):
    """Enhanced image loading with better preprocessing for SAR and optical data"""
    try:
        image_path = os.path.join(folder_path, f"{image_id}.npy")
        img = np.load(image_path).astype(np.float32)

        if len(img.shape) != 3 or img.shape[2] != 12:
            print(f"Invalid shape for {image_id}: {img.shape}")
            return None

        # Resize if necessary
        if img.shape[:2] != target_size:
            print(f"Resizing {image_id} from {img.shape[:2]} to {target_size}")
            resized_img = np.zeros((target_size[0], target_size[1], 12), dtype=np.float32)
            for channel in range(12):
                resized_img[:, :, channel] = tf.image.resize(
                    img[:, :, channel:channel+1], target_size
                ).numpy()[:, :, 0]
            img = resized_img

        processed_img = np.zeros_like(img, dtype=np.float32)

        # Process optical bands (0-3) with enhanced normalization
        for band in range(4):
            band_data = img[:, :, band]

            # Remove outliers using percentile clipping
            p2, p98 = np.percentile(band_data, [2, 98])
            band_data = np.clip(band_data, p2, p98)

            # Normalize to [0, 1]
            if band_data.max() > band_data.min():
                processed_img[:, :, band] = (band_data - band_data.min()) / (band_data.max() - band_data.min())
            else:
                processed_img[:, :, band] = 0.5

        # Process SAR bands (4-11) with improved handling
        for band in range(4, 12):
            sar_data = img[:, :, band]

            # Convert to dB scale with better handling of negative values
            sar_linear = np.abs(sar_data) + 1e-10
            sar_db = 10 * np.log10(sar_linear)

            # Clip extreme values
            p1, p99 = np.percentile(sar_db, [1, 99])
            sar_db = np.clip(sar_db, p1, p99)

            # Normalize
            if sar_db.max() > sar_db.min():
                processed_img[:, :, band] = (sar_db - sar_db.min()) / (sar_db.max() - sar_db.min())
            else:
                processed_img[:, :, band] = 0.5

        return processed_img

    except Exception as e:
        print(f"Error loading {image_id}: {str(e)}")
        return None

# Enhanced data generator with custom augmentation for multi-channel images
class EnhancedLandslideGenerator(Sequence):
    def __init__(self, image_ids, labels, folder_path, batch_size=16,
                 augment=False, shuffle=True, mixup_alpha=0.2):
        super().__init__()
        self.image_ids = image_ids
        self.labels = labels
        self.folder_path = folder_path
        self.batch_size = batch_size
        self.augment = augment
        self.shuffle = shuffle
        self.mixup_alpha = mixup_alpha

        # Validate files
        self.valid_indices = self._validate_files()
        print(f"Valid files: {len(self.valid_indices)}/{len(self.image_ids)}")

        self.on_epoch_end()

    def _validate_files(self):
        valid_indices = []
        for idx, img_id in enumerate(self.image_ids):
            file_path = os.path.join(self.folder_path, f"{img_id}.npy")
            if os.path.exists(file_path):
                try:
                    test_img = np.load(file_path)
                    if len(test_img.shape) == 3 and test_img.shape[2] == 12:
                        valid_indices.append(idx)
                except:
                    continue
        return valid_indices

    def __len__(self):
        return int(np.ceil(len(self.valid_indices) / self.batch_size))

    def custom_augment(self, img):
        """Custom augmentation for multi-channel images"""
        if not self.augment:
            return img

        # Random horizontal flip
        if np.random.random() < 0.5:
            img = np.flip(img, axis=1)

        # Random vertical flip
        if np.random.random() < 0.5:
            img = np.flip(img, axis=0)

        # Random rotation (90, 180, 270 degrees)
        if np.random.random() < 0.5:
            k = np.random.randint(1, 4)
            img = np.rot90(img, k=k, axes=(0, 1))

        # Random brightness adjustment (only for optical bands 0-3)
        if np.random.random() < 0.3:
            brightness_factor = np.random.uniform(0.8, 1.2)
            img[:, :, :4] = np.clip(img[:, :, :4] * brightness_factor, 0, 1)

        # Random noise addition
        if np.random.random() < 0.2:
            noise = np.random.normal(0, 0.01, img.shape).astype(np.float32)
            img = np.clip(img + noise, 0, 1)

        # Random translation
        if np.random.random() < 0.3:
            shift_x = np.random.randint(-20, 21)
            shift_y = np.random.randint(-20, 21)
            img = np.roll(img, shift_x, axis=1)
            img = np.roll(img, shift_y, axis=0)

        return img

    def mixup(self, x1, x2, y1, y2, alpha):
        """Apply mixup augmentation"""
        lam = np.random.beta(alpha, alpha)
        mixed_x = lam * x1 + (1 - lam) * x2
        mixed_y = lam * y1 + (1 - lam) * y2
        return mixed_x, mixed_y

    def __getitem__(self, idx):
        batch_indices = self.valid_indices[idx * self.batch_size:(idx + 1) * self.batch_size]

        batch_images = []
        batch_labels = []

        for i in batch_indices:
            img_id = self.image_ids[i]
            label = self.labels[i]

            img = load_and_preprocess_image(img_id, self.folder_path)
            if img is not None:
                # Apply custom augmentation instead of ImageDataGenerator
                img = self.custom_augment(img)
                batch_images.append(img)
                batch_labels.append(label)

        if len(batch_images) == 0:
            batch_images = [np.zeros((256, 256, 12), dtype=np.float32)]
            batch_labels = [0]

        batch_images = np.array(batch_images, dtype=np.float32)
        batch_labels = np.array(batch_labels, dtype=np.float32)

        # Apply mixup for training
        if self.augment and len(batch_images) >= 2 and np.random.random() < 0.3:
            indices = np.random.permutation(len(batch_images))
            mixed_images, mixed_labels = self.mixup(
                batch_images, batch_images[indices],
                batch_labels, batch_labels[indices],
                self.mixup_alpha
            )
            return mixed_images, mixed_labels

        return batch_images, batch_labels

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.valid_indices)

# Enhanced metrics
def balanced_f1_score(y_true, y_pred):
    """Calculate balanced F1 score considering both classes"""
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')

    # F1 for positive class
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))

    precision_pos = tp / (tp + fp + K.epsilon())
    recall_pos = tp / (tp + fn + K.epsilon())
    f1_pos = 2 * precision_pos * recall_pos / (precision_pos + recall_pos + K.epsilon())

    # F1 for negative class
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    precision_neg = tn / (tn + fn + K.epsilon())
    recall_neg = tn / (tn + fp + K.epsilon())
    f1_neg = 2 * precision_neg * recall_neg / (precision_neg + recall_neg + K.epsilon())

    # Return macro F1
    return (f1_pos + f1_neg) / 2

def precision_m(y_true, y_pred):
    y_pred = K.round(K.clip(y_pred, 0, 1))
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return tp / (predicted_positives + K.epsilon())

def recall_m(y_true, y_pred):
    y_pred = K.round(K.clip(y_pred, 0, 1))
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return tp / (possible_positives + K.epsilon())

# Enhanced loss functions
def focal_loss(gamma=2.0, alpha=0.75):
    def focal_loss_fixed(y_true, y_pred):
        y_true = K.cast(y_true, 'float32')
        y_pred = K.cast(K.clip(y_pred, K.epsilon(), 1 - K.epsilon()), 'float32')

        ce_loss = -y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        focal_weight = alpha * K.pow(1 - p_t, gamma)

        return K.mean(focal_weight * ce_loss)
    return focal_loss_fixed

def combined_loss(y_true, y_pred):
    """Combine focal loss with dice loss for better performance"""
    focal = focal_loss(gamma=2.0, alpha=0.75)(y_true, y_pred)

    # Dice loss component
    smooth = 1e-6
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice_coeff = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    dice_loss = 1 - dice_coeff

    return focal + 0.3 * dice_loss

# Enhanced model architecture with flexible input size
def create_enhanced_model(input_shape):
    """Create an enhanced CNN with attention mechanism"""
    print(f"Creating model with input shape: {input_shape}")
    inputs = Input(shape=input_shape)

    # Initial conv block
    x = Conv2D(32, (3, 3), padding='same', kernel_regularizer=l2(1e-4))(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Residual blocks with increasing complexity
    def residual_block(x, filters, downsample=False):
        shortcut = x
        stride = 2 if downsample else 1

        # Use regular Conv2D for better compatibility
        x = Conv2D(filters, (3, 3), strides=stride, padding='same',
                   kernel_regularizer=l2(1e-4))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(filters, (3, 3), padding='same',
                   kernel_regularizer=l2(1e-4))(x)
        x = BatchNormalization()(x)

        if downsample or shortcut.shape[-1] != filters:
            shortcut = Conv2D(filters, (1, 1), strides=stride, padding='same',
                             kernel_regularizer=l2(1e-4))(shortcut)
            shortcut = BatchNormalization()(shortcut)

        x = Add()([x, shortcut])
        x = Activation('relu')(x)
        x = Dropout(0.1)(x)

        return x

    # Adaptive feature extraction based on input size
    current_size = input_shape[0]  # Assume square images

    if current_size >= 256:
        # Full architecture for large images
        x = residual_block(x, 64, downsample=True)   # /2
        x = residual_block(x, 64)
        x = residual_block(x, 128, downsample=True)  # /4
        x = residual_block(x, 128)
        x = residual_block(x, 256, downsample=True)  # /8
        x = residual_block(x, 256)
        x = residual_block(x, 512, downsample=True)  # /16
    elif current_size >= 128:
        # Medium architecture
        x = residual_block(x, 64, downsample=True)   # /2
        x = residual_block(x, 128, downsample=True)  # /4
        x = residual_block(x, 256, downsample=True)  # /8
        x = residual_block(x, 512, downsample=True)  # /16
    else:
        # Smaller architecture for small images
        x = residual_block(x, 64, downsample=True)   # /2
        x = residual_block(x, 128, downsample=True)  # /4
        x = residual_block(x, 256)

    # Simplified attention mechanism (remove if causing issues)
    try:
        # Get current spatial dimensions
        spatial_dims = x.shape[1] * x.shape[2]
        channels = x.shape[3]

        # Only apply attention if we have reasonable dimensions
        if spatial_dims > 4 and spatial_dims < 1024:
            # Reshape for attention
            x_reshaped = Reshape((spatial_dims, channels))(x)

            # Multi-head attention with reduced complexity
            attention_output = MultiHeadAttention(
                num_heads=min(8, channels // 8),
                key_dim=min(64, channels // 4),
                dropout=0.1
            )(x_reshaped, x_reshaped)

            attention_output = LayerNormalization()(attention_output + x_reshaped)

            # Reshape back
            x_attended = Reshape((x.shape[1], x.shape[2], channels))(attention_output)

            # Combine original and attended features
            x = Concatenate()([x, x_attended])

    except Exception as e:
        print(f"Attention mechanism failed, using standard processing: {e}")
        # Fallback to standard processing
        pass

    # Final processing
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    x = Dense(256, activation='relu', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    outputs = Dense(1, activation='sigmoid', dtype='float32', name='predictions')(x)

    model = Model(inputs, outputs)
    return model

# Prepare data with stratified split
train_idx, val_idx = train_test_split(
    np.arange(len(train_df)),
    test_size=0.2,
    random_state=42,
    stratify=train_df['label']
)

# Calculate class weights
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_df['label']),
    y=train_df['label']
)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
print(f"Class weights: {class_weight_dict}")

# Create generators
train_gen = EnhancedLandslideGenerator(
    image_ids=train_df['ID'].values[train_idx],
    labels=train_df['label'].values[train_idx],
    folder_path=train_data_path,
    batch_size=8,  # Smaller batch for better convergence
    augment=True,
    mixup_alpha=0.2
)

val_gen = EnhancedLandslideGenerator(
    image_ids=train_df['ID'].values[val_idx],
    labels=train_df['label'].values[val_idx],
    folder_path=train_data_path,
    batch_size=8,
    augment=False
)

# Test a batch to check shapes
print("Testing data generators...")
try:
    test_batch_x, test_batch_y = train_gen[0]
    print(f"Train batch shape: {test_batch_x.shape}")
    print(f"Train labels shape: {test_batch_y.shape}")

    val_batch_x, val_batch_y = val_gen[0]
    print(f"Val batch shape: {val_batch_x.shape}")
    print(f"Val labels shape: {val_batch_y.shape}")

    # Update model input shape based on actual data
    actual_shape = test_batch_x.shape[1:]
    print(f"Creating model with input shape: {actual_shape}")

except Exception as e:
    print(f"Error testing generators: {e}")
    # Use default shape if testing fails
    actual_shape = (256, 256, 12)

# Create and compile model with correct input shape
model = create_enhanced_model(actual_shape)

# Use AdamW optimizer with cosine decay
initial_learning_rate = 1e-3
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=len(train_gen) * 30,
    alpha=1e-6
)

optimizer = AdamW(learning_rate=lr_schedule, weight_decay=1e-4)

model.compile(
    optimizer=optimizer,
    loss=combined_loss,
    metrics=['accuracy', precision_m, recall_m, balanced_f1_score]
)

print(f"Model parameters: {model.count_params():,}")

# Enhanced callbacks
callbacks = [
    ModelCheckpoint(
        filepath='best_landslide_model.keras',
        monitor='val_balanced_f1_score',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_balanced_f1_score',
        mode='max',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_balanced_f1_score',
        mode='max',
        factor=0.5,
        patience=7,
        min_lr=1e-7,
        verbose=1
    )
]

# Train model
print("Starting training...")
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=50,
    callbacks=callbacks,
    class_weight=class_weight_dict,
    verbose=1
)

# Load best model
model.load_weights('best_landslide_model.keras')

# Enhanced threshold optimization
print("Optimizing decision threshold...")
val_predictions = []
val_labels = []

for i in range(len(val_gen)):
    batch_x, batch_y = val_gen[i]
    pred_batch = model.predict(batch_x, verbose=0)
    val_predictions.extend(pred_batch.flatten())
    val_labels.extend(batch_y.flatten())

y_probs = np.array(val_predictions)
y_true = np.array(val_labels)

# Grid search for optimal threshold
thresholds = np.arange(0.1, 0.9, 0.005)
best_f1 = 0
best_thresh = 0.5
best_metrics = {}

for thresh in thresholds:
    y_pred = (y_probs > thresh).astype(int)

    # Calculate metrics for both classes
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_pos = f1_score(y_true, y_pred, pos_label=1)
    f1_neg = f1_score(y_true, y_pred, pos_label=0)

    # Optimize for balanced performance
    balanced_score = f1_macro * 0.6 + min(f1_pos, f1_neg) * 0.4

    if balanced_score > best_f1:
        best_f1 = balanced_score
        best_thresh = thresh
        best_metrics = {
            'f1_macro': f1_macro,
            'f1_positive': f1_pos,
            'f1_negative': f1_neg,
            'balanced_score': balanced_score
        }

print(f"\nOptimal threshold: {best_thresh:.3f}")
print(f"Best metrics: {best_metrics}")

# Final evaluation
y_pred_final = (y_probs > best_thresh).astype(int)
print(f"\nFinal Classification Report:")
print(classification_report(y_true, y_pred_final,
                          target_names=['No Landslide', 'Landslide']))

# Generate test predictions
print("Generating test predictions...")
test_df = pd.read_csv(test_csv)
test_predictions = []

batch_size = 16
for i in range(0, len(test_df), batch_size):
    batch_ids = test_df['ID'].values[i:i+batch_size]
    batch_imgs = []

    for img_id in batch_ids:
        img = load_and_preprocess_image(img_id, test_data_path)
        if img is not None:
            batch_imgs.append(img)
        else:
            batch_imgs.append(np.zeros((256, 256, 12), dtype=np.float32))

    batch_imgs = np.array(batch_imgs)
    probs = model.predict(batch_imgs, verbose=0).flatten()
    preds = (probs > best_thresh).astype(int)
    test_predictions.extend(preds)

# Create submission
submission_df = pd.DataFrame({
    'ID': test_df['ID'],
    'label': test_predictions
})

submission_df.to_csv('Enhanced_Landslide_Submission.csv', index=False)
print(f"Submission saved! Prediction distribution: {Counter(test_predictions)}")

# Cleanup
gc.collect()
print("Training completed successfully!")

Streaming output truncated to the last 5000 lines.
Resizing ID_51S3M4 from (64, 64) to (256, 256)
500/715 ━━━━━━━━━━━━━━━━━━━━ 1:59 555ms/step - accuracy: 0.7206 - balanced_f1_score: 0.4675 - loss: 0.6110 - precision_m: 1.2067 - recall_m: 1.1449Resizing ID_N3ZB67 from (64, 64) to (256, 256)
Resizing ID_Y20KZC from (64, 64) to (256, 256)
Resizing ID_8SAUMJ from (64, 64) to (256, 256)
Resizing ID_OARV3X from (64, 64) to (256, 256)
Resizing ID_ZZFZPA from (64, 64) to (256, 256)
Resizing ID_NE7IS5 from (64, 64) to (256, 256)
Resizing ID_R0N53D from (64, 64) to (256, 256)
Resizing ID_PU2BAY from (64, 64) to (256, 256)
501/715 ━━━━━━━━━━━━━━━━━━━━ 1:58 555ms/step - accuracy: 0.7206 - balanced_f1_score: 0.4675 - loss: 0.6110 - precision_m: 1.2069 - recall_m: 1.1453Resizing ID_CI00OX from (64, 64) to (256, 256)
Resizing ID_Q1BLPW from (64, 64) to (256, 256)
Resizing ID_SPQ8NS from (64, 64) to (256, 256)
Resizing ID_540P57 from (64, 64) to (256, 256)
Resizing ID_M0YJD0 from (64, 64) to (256, 25

KeyboardInterrupt: 